In [2]:
import pandas as pd
import pickle


In [3]:
def read_file(file, names):
    return pd.read_csv(file
            , sep=' \+\+\+\$\+\+\+ ' # solved delim issue must be regex 
            , header = None
            , lineterminator='\r\n'
            #, index_col=0
            , encoding='latin-1' # solved issue with loading
            , names=names
            )

# load conversation dataframes from txt format 
df_characters = read_file('movie_characters_metadata.txt',
                        names=['speaker_id','meta.character_name', 'meta.movie_idx', 'meta.movie_name','meta.gender', 'meta.credit_pos'])
df_conversations = read_file('movie_conversations.txt',
                        names=['speaker1','speaker2','meta.movie_id','conversation_lines'])
df_lines = read_file('movie_lines.txt',
                        names=['line_id','speaker','meta.movie_id','meta.character_name','text'])
df_metadata = read_file('movie_titles_metadata.txt',
                        names=['meta.movie_id','meta.movie_name', 'meta.release_year','meta.rating', 'meta.votes', 'meta.genre'])


In [4]:
# add release decade column
df_metadata['meta.release_decade'] = df_metadata['meta.release_year'].str[:3]
df_metadata['meta.release_decade'].value_counts()

199    244
200    144
198    108
197     51
196     19
195     17
193     16
194     15
192      2
201      1
Name: meta.release_decade, dtype: int64

In [5]:
### add missing genders
# load additional movie data. source: kaggle https://www.kaggle.com/rounakbanik/the-movies-dataset
credits = pd.read_csv('credits.csv')
# movie data corpus
movies_metadata = pd.read_csv('movies_metadata.csv')

# ensure numeric ids
credits["id"] = pd.to_numeric(credits["id"])
movies_metadata = movies_metadata[movies_metadata.id.apply(lambda x: x.isnumeric())]
movies_metadata["id"] = pd.to_numeric(movies_metadata["id"])
movies_metadata = pd.merge(movies_metadata, credits, left_on='id', right_on='id')

# evaluae json and make it parseable
from ast import literal_eval
movies_metadata['cast2'] = movies_metadata['cast'].apply(literal_eval)

#print(df_characters['meta.movie_idx'], df_metadata['meta.movie_id'])

# JOIN charactes df and metadata df.  
df_missing = pd.merge(  df_characters
                    , df_metadata[['meta.movie_id','meta.release_year']]
                    , left_on='meta.movie_idx'
                    , right_on='meta.movie_id'
                    , suffixes=("", "_m1")
                    )
# reduce dataframe to entries with missing gender
df_missing = df_missing[df_missing['meta.gender'] == '?' ]

# get unique movies by name and release year
df_missing_unique = df_missing[['meta.movie_name','meta.release_year']].drop_duplicates()
#print(df_missing_unique)

# mark movies in movies_metadata based on unique movies list as relevant to reduce data for next step
def get_sublist(x):
        movies_metadata.loc[
            movies_metadata['original_title'].str.contains(x['meta.movie_name'], case=False) 
            & movies_metadata['release_date'].str.contains(x['meta.release_year'], case=False)
            ,'relevant'] = 1
df_missing_unique.apply(get_sublist,axis=1)

# only relavant movies will 
movies_metadata_relevant = movies_metadata[movies_metadata['relevant'] == 1]


In [28]:

from tqdm import tqdm
import string

def get_characters_gender(name,x):
    for i in x:
        if name in i['character'].upper():
            if i['gender'] == 1:
               return 'f'
            elif i['gender'] == 2:
               return 'm'
    return '?'

def remove_punctuations(text):
    for punctuation in '"':
        text = text.replace(punctuation, '')
    return text

last_movie = ""

# iterate characters without gender information 
for index, row in tqdm(df_missing.iterrows()):
    if last_movie != row['meta.movie_name']: 
        #print(last_movie)
        #print(row['meta.character_name'])

        # find cast for a movie in the additional data set from kaggle using MOVIE NAME and RELEASE YEAR.
        cast2 = movies_metadata_relevant.query('original_title.str.contains("' + remove_punctuations(str(row['meta.movie_name'])) + '", case=False) and release_date.str.contains("' + row['meta.release_year'] + '", case=False) and cast.str.contains("' + remove_punctuations(str(row['meta.character_name'])) + '", case=False)', engine='python')['cast2']

    # find character gender in additional data and set it in working df
    for movie_casts in cast2:
        gen = get_characters_gender(row['meta.character_name'],movie_casts)
        df_characters.loc[df_characters['speaker_id'] == row['speaker_id'],'meta.gender'] = gen
        
    last_movie == row['meta.movie_name']

#df_missing[df_missing['meta.gender'] == '?' ]

6020it [02:57, 33.98it/s]


In [29]:
# Pickle it for later use
df_characters.to_pickle("df_characters.pkl")
df_conversations.to_pickle("df_conversations.pkl")
df_lines.to_pickle("df_lines.pkl")
df_metadata.to_pickle("df_metadata.pkl")